In [39]:
import sys
sys.path.append('../')
import pandas as pd
from Tools.plotting import CandlePlot
import numpy as np

In [40]:
config = {
    "risk_percent": 3,
    "tp_pct": 0.4,  # TP as % of price for crypto
    "sl_pct": 0.4,  # SL as % of price for crypto
    'sl_tp_dist': 0.005,
    # "expiration_bars": 100,  # how many bars to wait before expiring an order
    "bars_n": 11,  # the look-back period for finding highs/lows
}

In [41]:
# Load the data and convert the timestamp to datetime
df = pd.read_csv('../DB/5m/INJUSDT_5m.csv', parse_dates=True) 

# df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
# # # Convert 'date' to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])  # Assuming the 'date' is in milliseconds

# # # Set 'date' as index
df.set_index('datetime', inplace=True)

# df.iloc[0]
# df.dtypes
# df.head()
df= df[:].copy()

In [42]:
def find_high_low(candle, window=5):
    
    PIVOT_H = 1
    PIVOT_L = -1
    if ((candle-window) <0) | ((candle+window)>=len(df)):
        return 0
    
    pivot_high = True
    pivot_low = True
    
    for i in range(candle - window, candle + window + 1):
        if df.iloc[candle].high < df.iloc[i].high:
            pivot_high = False
        if df.iloc[candle].low > df.iloc[i].low:
            pivot_low = False
    
    if pivot_high and pivot_low:
        return 0
    elif pivot_high:
        return PIVOT_H
    elif pivot_low:
        return PIVOT_L
    else:
        return 0  # Not a pivot point
    


In [43]:
df = df.reset_index()
df['hh_ll'] = df.apply(lambda row: find_high_low(row.name), axis=1)

In [44]:
df.set_index('datetime', inplace=True)

df



,open,high,low,close,volume,quote_volume,hh_ll
datetime,,,,,,,
2022-11-21 03:15:00,1.564,1.564,1.559,1.559,0,0,0
2022-11-21 03:20:00,1.559,1.560,1.556,1.557,0,0,0
2022-11-21 03:25:00,1.557,1.564,1.553,1.563,0,0,0
2022-11-21 03:30:00,1.563,1.576,1.563,1.576,0,0,0
2022-11-21 03:35:00,1.576,1.577,1.575,1.577,0,0,0
...,...,...,...,...,...,...,...
2024-09-04 04:05:00,15.645,15.651,15.603,15.608,0,0,0
2024-09-04 04:10:00,15.608,15.630,15.600,15.627,0,0,0
2024-09-04 04:15:00,15.627,15.631,15.572,15.576,0,0,0


In [45]:
def plot(df, traces=[], points=[], win=-2000):
        dfplt = df[win:win+1000]
        cp = CandlePlot(dfplt)
            # Find the date range in dfplt
        start_date = dfplt.index.min()  # assuming dfplt index is datetime
        end_date = dfplt.index.max()
        # Filter trades within the date range of dfplt
        cp.add_traces(traces)
        if points is not None:
            cp.add_points(points)
        cp.show_plot()

In [46]:

# plot(df, [], 'hh_ll', -2000)

In [47]:
last_h_l = {"hh": None, "ll": None}

for index, row in df.iterrows():
    # Update the last higher-high (HH) and lower-low (LL) if there's a pivot high or low
    if row['hh_ll'] == 1:  # New pivot high
        last_h_l['hh'] = row['high']
    elif row['hh_ll'] == -1:  # New pivot low
        last_h_l['ll'] = row['low']

    # Store the last HH and LL in the data frame
    df.loc[index, 'hh'] = last_h_l['hh']
    df.loc[index, 'll'] = last_h_l['ll']
   
        

In [48]:

# plot(df, ['ll', 'hh'],'hh_ll', -2000)

In [49]:
BUY = 1
SELL = -1
NONE = 0
open_positions = {"buy": None, "sell": None} 
df['SIGNAL'] = NONE

In [50]:
current_position = 0 

for index, row in df.iterrows():
    if (current_position !=BUY) and (row['hh'] is not None) and (row['close'] >= row['hh']): #-row['close'] * config['sl_tp_dist']
        df.loc[index, 'SIGNAL'] = BUY
        current_position = BUY
    elif (current_position !=SELL) and (row['ll'] is not None) and (row['close'] <= row['ll']): #+row['close'] * config['sl_tp_dist']
        df.loc[index, 'SIGNAL'] = SELL
        current_position = SELL
    


In [51]:
# plot(df, ['ll', 'hh'], 'SIGNAL', -2000)

In [52]:
df

,open,high,low,close,volume,quote_volume,hh_ll,hh,ll,SIGNAL
datetime,,,,,,,,,,
2022-11-21 03:15:00,1.564,1.564,1.559,1.559,0,0,0,None,None,0
2022-11-21 03:20:00,1.559,1.560,1.556,1.557,0,0,0,None,None,0
2022-11-21 03:25:00,1.557,1.564,1.553,1.563,0,0,0,None,None,0
2022-11-21 03:30:00,1.563,1.576,1.563,1.576,0,0,0,None,None,0
2022-11-21 03:35:00,1.576,1.577,1.575,1.577,0,0,0,None,None,0
...,...,...,...,...,...,...,...,...,...,...
2024-09-04 04:05:00,15.645,15.651,15.603,15.608,0,0,0,15.683,15.51,0
2024-09-04 04:10:00,15.608,15.630,15.600,15.627,0,0,0,15.683,15.51,0
2024-09-04 04:15:00,15.627,15.631,15.572,15.576,0,0,0,15.683,15.51,0


In [57]:
def trail_stop(current_position, row, trailing_sl_trigger_pct, trailing_sl_pct):
    """
    Adjust the trailing stop loss based on price movement for crypto.
    
    current_position: dict, the current open position details
    row: pandas Series, current row of price data (includes 'high', 'low', 'close')
    trailing_sl_trigger_pct: float, the percentage at which the trailing stop is triggered
    trailing_sl_pct: float, the percentage distance for the trailing stop
    """
    sl = current_position['sl']  # Existing stop loss
    entry_price = current_position['entry_price']  # Entry price of the position

    # Trailing stop for BUY position
    if current_position['type'] == 'BUY':
        bid = row['close']  # Use the close price as bid price for simplicity
        if (bid - entry_price) > (trailing_sl_trigger_pct * entry_price):  # Trigger condition
            new_sl = bid - (trailing_sl_pct * entry_price)  # Calculate new stop loss
            if new_sl > sl:  # Only update if the new stop loss is higher (to protect profits)
                current_position['sl'] = new_sl  # Update the stop loss

    # Trailing stop for SELL position
    elif current_position['type'] == 'SELL':
        ask = row['close']  # Use the close price as ask price for simplicity
        if (entry_price - ask) > (trailing_sl_trigger_pct * entry_price):  # Trigger condition
            new_sl = ask + (trailing_sl_pct * entry_price)  # Calculate new stop loss
            if new_sl < sl:  # Only update if the new stop loss is lower (to protect profits)
                current_position['sl'] = new_sl  # Update the stop loss

    return current_position


In [59]:
trades = []  # Store trade details
current_position = {}  # Holds open position data
wallet = 1000  # Initial capital in USD
fee = 0.007  # Trading fee (0.7%)
risk = 0.03  # Risk per trade (3% of wallet)
sl_pct = 0.04  # Initial stop-loss percentage (1%)
tp_pct = 0.04  # Initial take-profit percentage (3%)
trailing_sl_pct = 0.005  # Trailing stop percentage (0.5%)
trailing_tp_pct = 0.005  # Trailing take-profit percentage (0.5%)

for index, row in df.iterrows():
    # Check if there's no open position
    if not current_position:
        
        # Check for a BUY signal
        if row['SIGNAL'] == BUY:
            # Position size is based on the risk amount divided by close price
            pos_size = (risk * wallet) / row['close']
            entry_price = row['close']
            pos_fee = pos_size * entry_price * fee
            wallet -= pos_fee  # Deduct fee from wallet
            entry_date = index

            sl = entry_price * (1 - sl_pct)  # Stop-loss for BUY
            tp = entry_price * (1 + tp_pct)  # Take-profit for BUY

            # Store the open position details
            current_position = {
                'entry_date': entry_date,
                'type': 'BUY',
                'size': pos_size,
                'entry_price': entry_price,
                'sl': sl,
                'tp': tp,
                'trailing_sl': sl,  # Initial trailing stop
                'trailing_tp': tp  # Initial trailing take-profit
            }

        # Check for a SELL signal
        elif row['SIGNAL'] == SELL:
            # Position size is based on the risk amount divided by close price
            pos_size = (risk * wallet) / row['close']
            entry_price = row['close']
            pos_fee = pos_size * entry_price * fee
            wallet -= pos_fee  # Deduct fee from wallet
            entry_date = index

            sl = entry_price * (1 + sl_pct)  # Stop-loss for SELL
            tp = entry_price * (1 - tp_pct)  # Take-profit for SELL

            # Store the open position details
            current_position = {
                'entry_date': entry_date,
                'type': 'SELL',
                'size': pos_size,
                'entry_price': entry_price,
                'sl': sl,
                'tp': tp,
                'trailing_sl': sl,  # Initial trailing stop
                'trailing_tp': tp  # Initial trailing take-profit
            }

    # If there is an open position, check for close or trail the stop/TP
    else:
        # Apply trailing stop logic
        current_position = trail_stop(current_position, row, trailing_sl_trigger_pct=0.01, trailing_sl_pct=0.005)
        if current_position['type'] == 'BUY':
            # Check if price has hit stop-loss
            if row['low'] <= current_position['sl']:
                pnl = (current_position['sl'] - current_position['entry_price']) * current_position['size'] - pos_fee
                wallet += current_position['sl'] * current_position['size'] - pos_fee
                trades.append({'type': current_position['type'], 
                               'entry_date': current_position['entry_date'],
                               'exit_date': index,
                               'entry_price': current_position['entry_price'], 
                               'exit_price': current_position['sl'], 
                               'pnl': pnl,
                               'exit_reason': 'SL'
                               })
                current_position = {}  # Close position

            # Check if price has hit take-profit
            elif row['high'] >= current_position['tp']:
                pnl = (current_position['tp'] - current_position['entry_price']) * current_position['size'] - pos_fee
                wallet += current_position['tp'] * current_position['size'] - pos_fee
                trades.append({'type': current_position['type'], 
                               'entry_date': current_position['entry_date'],
                               'exit_date': index, 
                               'entry_price': current_position['entry_price'], 
                               'exit_price': current_position['tp'], 
                               'pnl': pnl,
                               'exit_reason': 'TP'
                               })
                current_position = {}  # Close position

            
        elif current_position['type'] == 'SELL':
            # Check if price has hit stop-loss
            if row['high'] >= current_position['sl']:
                pnl = (current_position['entry_price'] - current_position['sl']) * current_position['size'] - pos_fee
                wallet += (current_position['entry_price'] - current_position['sl']) * current_position['size'] - pos_fee
                trades.append({'type': current_position['type'], 
                               'entry_date': current_position['entry_date'],
                               'exit_date': index,  
                               'entry_price': current_position['entry_price'], 
                               'exit_price': current_position['sl'], 
                               'pnl': pnl,
                               'exit_reason': 'SL'
                               })
                current_position = {}  # Close position

            # Check if price has hit take-profit
            elif row['low'] <= current_position['tp']:
                pnl = (current_position['entry_price'] - current_position['tp']) * current_position['size'] - pos_fee
                wallet += (current_position['entry_price'] - current_position['tp']) * current_position['size'] - pos_fee
                trades.append({'type': current_position['type'], 
                               'entry_date': current_position['entry_date'],
                               'exit_date': index, 
                               'entry_price': current_position['entry_price'], 
                               'exit_price': current_position['tp'], 
                               'pnl': pnl,
                               'exit_reason': 'TP'
                               })
                current_position = {}  # Close position

            
# Final wallet value
print(f"Final wallet value: {wallet}")

# Print trades for review
for trade in trades:
    print(trade)


Final wallet value: 85838670477276.1
{'type': 'BUY', 'entry_date': Timestamp('2022-11-21 05:00:00'), 'exit_date': Timestamp('2022-11-21 13:20:00'), 'entry_price': 1.593, 'exit_price': 1.608035, 'pnl': 0.07314500941619836, 'exit_reason': 'SL'}
{'type': 'SELL', 'entry_date': Timestamp('2022-11-21 14:30:00'), 'exit_date': Timestamp('2022-11-21 16:30:00'), 'entry_price': 1.587, 'exit_price': 1.564935, 'pnl': 0.21329263925752676, 'exit_reason': 'SL'}
{'type': 'BUY', 'entry_date': Timestamp('2022-11-21 19:15:00'), 'exit_date': Timestamp('2022-11-21 21:40:00'), 'entry_price': 1.554, 'exit_price': 1.56823, 'pnl': 0.06664268875932977, 'exit_reason': 'SL'}
{'type': 'SELL', 'entry_date': Timestamp('2022-11-21 23:25:00'), 'exit_date': Timestamp('2022-11-22 02:40:00'), 'entry_price': 1.564, 'exit_price': 1.53182, 'pnl': 0.43194617424138354, 'exit_reason': 'SL'}
{'type': 'BUY', 'entry_date': Timestamp('2022-11-22 05:10:00'), 'exit_date': Timestamp('2022-11-22 12:55:00'), 'entry_price': 1.51, 'exit_p

In [54]:
# def trailing_stop(position, current_price, config):
#     if position == "buy":
#         if current_price - open_positions["buy"]["entry"] > config['tsl_trigger_points']:
#             new_sl = current_price - config['tsl_points']
#             open_positions["buy"]["sl"] = max(open_positions["buy"]["sl"], new_sl)

#     elif position == "sell":
#         if open_positions["sell"]["entry"] - current_price > config['tsl_trigger_points']:
#             new_sl = current_price + config['tsl_points']
#             open_positions["sell"]["sl"] = min(open_positions["sell"]["sl"], new_sl)


In [55]:
# # Track the time when an order was placed
# order_time = df['timestamp'][index]

# # Later, compare the current time with order_time
# if (current_time - order_time).seconds > config['expiration_bars'] * bar_seconds:
#     close_order()  # Close the order if expired
